In [1]:
!pip install -q gradio &> /dev/null
!pip install langchain &> /dev/null
!pip install openai &> /dev/null
!pip install google-search-results &> /dev/null
!pip install python-dotenv &> /dev/null
!pip -q install unstructured chromadb tiktoken  &> /dev/null

In [2]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma
import os
import shutil

openai_api_key=os.getenv('OPENAI_API_KEY', '')


CHROMA_PATH = "chroma"
DATA_PATH = "./data/books"
!mkdir -p ./data/books
!gsutil cp gs://2022_chembdrc/my_space/*.md ./data/books/
# !rm ./data/books/SocialAnxiety.md
# !rm ./data/books/Shynes,SocialAnxietyWorkbook.md

def main():
    generate_data_store()


def generate_data_store():
    documents = load_documents()
    chunks = split_text(documents)
    save_to_chroma(chunks)


def load_documents():
    loader = DirectoryLoader(DATA_PATH, glob="*.md")
    documents = loader.load()
    return documents


def split_text(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    document = chunks[10]
    print(document.page_content)
    print(document.metadata)

    return chunks


def save_to_chroma(chunks: list[Document]):
    # Clear out the database first.
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

    # Create a new DB from the documents.
    db = Chroma.from_documents(
        chunks, OpenAIEmbeddings(openai_api_key=openai_api_key), persist_directory=CHROMA_PATH
    )
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")


if __name__ == "__main__":
    main()


Copying gs://2022_chembdrc/my_space/ShynessSocialAnxietyWorkbook.md...
Copying gs://2022_chembdrc/my_space/SocialAnxiety.md...
/ [2 files][596.2 KiB/596.2 KiB]                                                
Operation completed over 2 objects/596.2 KiB.                                    


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Split 2 documents into 3263 chunks.
Social anxiety is one of the most common sorts of anxiety and
affects both men and women. Most of us feel shy or anxious in
social situations at some point. This can be much more serious
for some people, however, who find that their relationships,
{'source': 'data/books/SocialAnxiety.md', 'start_index': 1748}


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Saved 3263 chunks to chroma.


In [4]:
import gradio as gr
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from dotenv import load_dotenv
import os
import argparse
from dataclasses import dataclass
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

load_dotenv()

# openai_api_key=os.getenv('OPENAI_API_KEY', 'sk-')
openai_api_key=os.getenv('OPENAI_API_KEY', 'sk-')
# sk-

# Initialize ChatOpenAI with desired temperature and your OpenAI API key
chat = ChatOpenAI(temperature=0.9, openai_api_key=openai_api_key)


def get_rag_advice(query_text) :
  CHROMA_PATH = "chroma"

  PROMPT_TEMPLATE = """
  Given the context of this literature :

  {context}

  ---

  Considering the specific concern expressed: "{question}", what advice would this literature suggest? Please provide practical steps and insights based on the context.
  """


  # Prepare the DB.
  embedding_function = OpenAIEmbeddings(openai_api_key=openai_api_key)
  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

  # Search the DB.
  results = db.similarity_search_with_relevance_scores(query_text, k=3)
  if len(results) == 0 or results[0][1] < 0.7:
      print(f"Unable to find matching results.")

  context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
  prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
  prompt = prompt_template.format(context=context_text, question=query_text)

  model = ChatOpenAI(openai_api_key=openai_api_key)
  response_text = model.predict(prompt)

  sources = [doc.metadata.get("source", None) for doc, _score in results]
  formatted_response = f"{response_text}\nSources: {sources}"
  return formatted_response



# Define the function to assess social anxiety levels and give advice
def social_anxiety_assessment(q1, q2, q3, q4, q5, q6, q7, q8, q9, q10):
    # Convert question responses to a list
    responses = [q1, q2, q3, q4, q5, q6, q7, q8, q9, q10]
    score = sum(responses)

    questions = [
    "I am afraid of people in authority.",
    "I am bothered by blushing in front of people.",
    "Parties and social events scare me.",
    "I avoid talking to people I don’t know.",
    "Being criticized scares me a lot.",
    "I avoid doing things or speaking to people for fear of embarrassment.",
    "Sweating in front of people causes me distress.",
    "I avoid going to parties.",
    "I avoid activities in which I am the center of attention.",
    "Talking to strangers scares me."
    ]

    # Filter and create a list of checked questions
    checked_questions = [questions[i] for i, response in enumerate(responses) if response]
    checked_questions_context = ", ".join(checked_questions)

    # Define result categories
    if score <= 3:
        category = "Normal"
    elif 4 <= score <= 7:
        category = "High"
    else:
        category = "Very high"

    # Update context to include checked questions
    context = (
        "The checked questions are "
        + checked_questions_context +
        ". Depending on the score, the categories are 'Normal', 'High', and 'Very High'."
    )

    # Using Langchain's ChatOpenAI with SystemMessage for context
    responses = chat([
        SystemMessage(content=context),
        HumanMessage(content=f"Given the checked questions and a score of {score}, which falls into the '{category}' category, Please provide advice for every checkecd questions. based on the latest psychiatric research findings on social anxiety." )
    ])

    # Extracting the AI's advice from the response
    advice = responses.content  # Assuming the response is in the first element
    advice += "\n\n======================================================================================================\n\n Advice from References : " + get_rag_advice(checked_questions_context)

    return f"Your anxiety level is: {category}\n\nAdvice: {advice}\n"

# List of questions
questions = [
    "I am afraid of people in authority.",
    "I am bothered by blushing in front of people.",
    "Parties and social events scare me.",
    "I avoid talking to people I don’t know.",
    "Being criticized scares me a lot.",
    "I avoid doing things or speaking to people for fear of embarrassment.",
    "Sweating in front of people causes me distress.",
    "I avoid going to parties.",
    "I avoid activities in which I am the center of attention.",
    "Talking to strangers scares me."
]

# Your existing Gradio interface code here
iface = gr.Interface(
    fn=social_anxiety_assessment,
    inputs=[gr.Checkbox(label=q) for q in questions],  # Removed value=False, as it's default
    outputs="text",
    title="Social Anxiety Assessment",
    description="This assessment is based on common symptoms of social anxiety. Select 'True' if the statement applies to you, otherwise leave it unchecked."
)

iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://66838363e967e81ec3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
